In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import r2_score

%matplotlib inline

Importing plotly failed. Interactive plots will not work.


In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df.head()

,periode,cash_in_echannel,cash_out_echannel,cash_in_kantor,cash_out_kantor,cr_ketetapan_total_bkn_sum,giro,deposito,kewajiban_lain,tabungan,rata_dpk_mingguan,kas_kantor,kas_echannel
0,2019-07-31,7.303000e+08,-1.304400e+09,1.436722e+11,-1.106104e+11,3.0,9.867358e+11,8.048153e+11,1.419685e+10,7.072647e+11,3.135744e+11,1.928940e+09,2.939100e+09
1,2019-08-01,7.322000e+08,-8.321500e+08,3.144131e+11,-6.710987e+10,3.0,8.962459e+11,8.125611e+11,1.234062e+10,7.011995e+11,3.135744e+11,2.492322e+11,2.839150e+09
2,2019-08-02,1.169800e+09,-6.214000e+08,1.251294e+09,-1.142332e+09,3.0,9.059714e+11,8.127225e+11,1.182022e+10,6.922787e+11,3.135744e+11,2.493411e+11,3.387550e+09
3,2019-08-03,9.134500e+08,-4.240500e+08,0.000000e+00,0.000000e+00,3.0,9.057127e+11,8.127253e+11,1.199640e+10,6.867224e+11,3.135744e+11,2.493411e+11,3.876950e+09
4,2019-08-04,7.752500e+08,-7.779500e+08,9.883331e+10,-8.729274e+10,3.0,9.788347e+11,8.124711e+11,1.232962e+10,6.813438e+11,3.135744e+11,2.608817e+11,3.874250e+09


In [4]:
train_df['periode'] = pd.to_datetime(train_df['periode'])
train_df = train_df.set_index('periode')

print(train_df.iloc[:3, :3])
print()
print(train_df.iloc[-3:, :3])

            cash_in_echannel  cash_out_echannel  cash_in_kantor
periode                                                        
2019-07-31      7.303000e+08      -1.304400e+09    1.436722e+11
2019-08-01      7.322000e+08      -8.321500e+08    3.144131e+11
2019-08-02      1.169800e+09      -6.214000e+08    1.251294e+09

            cash_in_echannel  cash_out_echannel  cash_in_kantor
periode                                                        
2020-09-28      7.438000e+08       -556200000.0    1.264345e+11
2020-09-29      1.009250e+09       -772100000.0    1.548627e+11
2020-09-30      6.861000e+08       -833250000.0    1.487817e+11


In [ ]:
train_df.info()

In [ ]:
import matplotlib.dates as mdates

In [ ]:
important_cols = ['kas_kantor', 'cash_in_kantor', 'cash_out_kantor', 'kas_echannel', 'cash_in_echannel', 'cash_out_echannel']

for col in important_cols:
    plt.figure(figsize=(20, 5))
    plt.plot(train_df[col])
    plt.grid(color='grey')
    plt.title(col)
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    plt.show()

In [ ]:
kas_kantor = train_df['kas_kantor']
kas_ec = train_df['kas_echannel']

X = np.arange(425).reshape(-1, 1)

lr_k = Prophet(changepoint_prior_scale=0.045, daily_seasonality=False)
lr_e = Prophet(changepoint_prior_scale=0.045, daily_seasonality=False)

lr_k.fit(X, kas_kantor)
lr_e.fit(X, kas_ec)

In [ ]:
x_pred = np.arange(425, 425 + 31).reshape(-1, 1)
yk_pred = lr_k.predict(x_pred)
ye_pred = lr_e.predict(x_pred)

plt.figure(figsize=(16,4))
plt.plot(kas_kantor)
plt.plot(yk_pred)
plt.grid(color='grey')
plt.title('kas kantor')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.show()

plt.figure(figsize=(16,4))
plt.plot(kas_ec)
plt.plot(ye_pred)
plt.grid(color='grey')
plt.title('kas echannel')
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.show()

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.iloc[:31, 1] = forecast_k.iloc[425:,3]
submission.iloc[31:, 1] = forecast_e.iloc[425:,3]
submission.to_csv('submission_x.csv', index=False)